In [112]:
import random as rnd
import numpy as np
import random


class Agent():
    def __init__(self,network,Z_func, id):
        self.strategy = None # A or B
        self.next_strategy = None
        self.previous_strategy = None
        self.neighbors_id = []
        self.Z_func = Z_func
        self.network = network
        self.type = None #=type = update rule(coordinating , anti_coordinating
        self.id = id
   

    def __coordinating(self, agents,Z_func,threshold):
        A_neighbors_count = 0
        B_neighbors_count = 0
        total_neighbors_count = len(self.neighbors_id)
        neighbors = []
        for neighbor_id in self.neighbors_id:
            if agents[neighbor_id].strategy =="A":
                A_neighbors_count= A_neighbors_count+1
            else :
                B_neighbors_count = B_neighbors_count+1
        # if self.binary_threshold =="normal":
#         print("co: "+str(A_neighbors_count)+" As and "+str(B_neighbors_count)+" Bs")
        if A_neighbors_count >= B_neighbors_count:
            self.next_strategy = "A"
        else:
            self.next_strategy = "B"
   
    def __anti_coordinating(self, agents,Z_func,threshold):
        A_neighbors_count = 0
        B_neighbors_count = 0
        total_neighbors_count = len(self.neighbors_id)
        for neighbor_id in self.neighbors_id:
            if agents[neighbor_id].strategy =="A":
                A_neighbors_count= A_neighbors_count+1
            else :
                B_neighbors_count = B_neighbors_count+1
        # if self.binary_threshold == "normal":
#         print("anti: "+str(A_neighbors_count)+" As and "+str(B_neighbors_count)+" Bs")
        if A_neighbors_count <= B_neighbors_count:
            self.next_strategy = "A"
        else:
            self.next_strategy = "B"

    def decide_next_strategy(self, agent,Z_func,threshold):
        if self.type == "+":
            self.__coordinating(agent,Z_func,threshold)

        elif self.type == "-":
            self.__anti_coordinating(agent,Z_func,threshold)

    def update_strategy(self):
        self.strategy = self.next_strategy

    

In [113]:
import numpy as np
import random as rnd
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
from csv import writer
import itertools


def plus(idd, number, population):
    if idd + number >= population:
        return idd + number - population - 1
    else:
        return idd + number


def minus(idd, number, population):
    if idd + number <= 0:
        return population - idd
    else:
        return idd - 1


class Simulation:
    def __init__(self, population, z_func, strategy_list, type_list, activated_list, desired_eq):
        self.network = None
        self.Z_func = z_func
        self.population = population
        self.strategy_list = strategy_list
        self.type_list = type_list
        self.activated_list = activated_list
        self.desired_eq = desired_eq
        self.agents = self.__generate_agents()

    def __generate_agents(self):
        self.network = nx.circulant_graph(self.population, [1])  # ring only
        agents = [Agent(self.network, self.Z_func, i) for i in range(self.population)]
        for index, focal in enumerate(agents):
            neighbors_id = list(self.network[index])
#             print(neighbors_id)
            for nb_id in neighbors_id:
                focal.neighbors_id.append(nb_id)
        return agents

    def __initialize_label_A_or_B(self):
        for index, focal in enumerate(self.agents):
            focal.strategy = self.strategy_list[index]

    def determine_coordinator_or_anticoordinator(self, co_list):
        for index, focal in enumerate(self.agents):
            if self.type_list[index] == '+':
                focal.type = "+"
            else:
                focal.type = "-"

    def __take_snapshot(self, timestep, equilibrated):

        for index, focal in enumerate(self.agents):
            if focal.strategy == "A":
                self.network.nodes[index]["strategy"] = "A"
            else:
                self.network.nodes[index]["strategy"] = "B"

        def color(i):
            if self.network.nodes[i]["strategy"] == "A":
                return 'cyan'
            else:
                return 'pink'

        color = dict((i, color(i)) for i in self.network.nodes())
        # if self.network_type == "ring":
        pos = nx.circular_layout(self.network)
        #
        # else:
        #     pos = nx.spring_layout(self.network)

        nx.draw_networkx_edges(self.network, pos)
        nx.draw_networkx_nodes(self.network, pos, node_color=list(color.values()), node_size=500)
        labels = {}
        for index, focal in enumerate(self.agents):
            if focal.type == "+":
                labels[index] = f"{index},+"
            else:
                labels[index] = f"{index},-"

        nx.draw_networkx_labels(self.network, pos, labels, font_size=16)
        eq_str = ""
        if equilibrated:
            eq_str = "Equilibrated!"
        plt.title(f"t={timestep}  {eq_str}", fontsize=20)
        plt.axis = "off"
        time_past_eq = -1

        plt.xticks([])
        plt.yticks([])
        plt.savefig(f"snap_t={timestep}.png", dpi=300)
        plt.close()

    def has_equilibrated(self):
        equilibrated = 1
        for index, focal in enumerate(self.agents):
            if focal.strategy != focal.previous_strategy:
                equilibrated = 0
                break

        return equilibrated

    def one_episode(self, time_steps, threshold):
        reached_desired = 0
        self.__initialize_label_A_or_B()
        self.determine_coordinator_or_anticoordinator(self.type_list)
        equilibrated = -1
        equilibrated_array = []
        current_strategy_list = self.strategy_list
        prev_i = -2
        for t in range(time_steps):
#             for f in self.agents:
#                 print("id: "+ str(f.id)+" type: "+f.type+" strategy: "+f.strategy+" willing to be: "+str(f.next_strategy))
            for i in range(self.population):
                self.agents[i].previous_strategy = self.agents[i].strategy
                # if(t == time_steps-1):
                
                (self.agents[i]).decide_next_strategy(self.agents, self.Z_func, threshold)
#                 print("next strategy in this timestep is decided for everyone!")
                current_strategy_list[i] = self.agents[i].strategy
                reached_desired = (self.strategy_list == current_strategy_list)
            # RANDOM AGENT TO BE ACTIVATED
            # index_list = rnd.sample(range(self.population), k = 1)#########GGG#####
            # index = index_list[0]
            # AGENT TO BE ACTIVATED UNDER OUR POLICY:
            index = self.activate_policy(prev_i)
            prev_i = index
            print("in timestep "+str(t)+" selected: agent "+str(index))
            
            # if index!=0 and index!= self.population-1:
            if index != -1:
                # for index in self.cooperators: (everybody is cooperating)
#                 (self.agents[index]).decide_next_strategy(self.agents, self.Z_func, threshold)
#                 print("selected agent next strategy: "+self.agents[index].next_strategy )
                # for index in self.cooperators:
                (self.agents[index]).update_strategy()
                # self.agents[index].next_strategy = (self.agents[index]).strategy
            else:
                print ("no one found\n")
                if self.has_equilibrated():
                    break
#                 else:
#                 self.activated_list = np.zeros(self.population)
            equilibrated = self.has_equilibrated()
            self.__take_snapshot(t, equilibrated)
            print("selected agent : "+str(index))
            equilibrated_array.append(equilibrated)
        eq_time = -1

        new_result = pd.DataFrame(
            {'reached_desired?': [reached_desired], 'Equilibrated?': [equilibrated], 'types': [self.type_list],
             'initial strategy set': [self.strategy_list], 'final strategy set': [current_strategy_list],
             'equilibration time': [eq_time]})
        return new_result, equilibrated, eq_time, reached_desired

    def activate_policy(self,prev_i):
        c1 = []
        c2 = []
        c3 = []
        for agent in self.agents:
            w = (agent.strategy == self.desired_eq[agent.id])
            B = (agent.strategy != agent.next_strategy)
#             print("w="+str(w),"B = "+str(B))
            if B == 1:
                if agent.strategy == 'B' and agent.type == '+' \
                        and (self.strategy_list[plus(agent.id, 1, self.population)] == 'A' \
                             or self.strategy_list[
                                 minus(agent.id, 1, self.population)] == 'A') and w==0:  # if agent[i+1]= A or agent[i-1] = A:
                    c3.append(agent.id)  # add agent i to c3
                elif (agent.strategy == 'B') and \
                     ((self.strategy_list[plus(agent.id, 1, self.population)] == 'B' and \
                       self.type_list[plus(agent.id, 1, self.population)] == '-' and \
                       self.strategy_list[plus(agent.id, 2, self.population)] == 'A' and \
                       self.type_list[plus(agent.id, 2, self.population)] == '-') or \
                      (self.strategy_list[minus(agent.id, 1, self.population)] == 'B' and \
                       self.type_list[minus(agent.id, 1, self.population)] == '-' and \
                       self.strategy_list[minus(agent.id, 2, self.population)] == 'A' and \
                       self.type_list[minus(agent.id, 2,
                       self.population)] == '-')) and w==0:  # else if (agent[i+1] = B- and agent[i+2] = A-) or (agent[i-1] = B- and agent[i-2] = A-):
                    c3.append(agent.id)  # add agent i to c3
                elif (agent.strategy == 'A' and agent.type == '-') and \
                        ((self.strategy_list[plus(agent.id, 1, self.population)] == 'A' and \
                          self.type_list[plus(agent.id, 1, self.population)] == '-') or \
                         (self.strategy_list[minus(agent.id, 1, self.population)] == 'A' and \
                          self.type_list[minus(agent.id, 1, self.population)] == '-')) and w==0:
                    c3.append(agent.id)  # else if agent i = A- and(agent i+1 = A- or agent i-1 = A-)
                    # add agent i to c3
                else:
                    if w == 0:
                        c1.append(agent.id)
                    else:
                        c2.append(agent.id)
        print("c1: "+str(c1)+" c2: "+str(c2)+" c3: "+str(c3))
        print("agents activated so far: "+ str(self.activated_list))
        for i in range(self.population):
            if i in c1 and i!= prev_i:
                self.activated_list[i] =self.activated_list[i]+ 1
                return i
        for i in range(self.population):
            if i in c2 and self.activated_list[i] <= 2 and i!= prev_i:
                self.activated_list[i] =self.activated_list[i]+ 1
                return i
        for i in range(self.population):
            if i in c3 and i!= prev_i:
                self.activated_list[i] =self.activated_list[i]+ 1
                return i
            
        return -1


In [114]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from numpy import zeros
import random
import  pandas as pd
import itertools
from pandas import concat


def main():
    population = 9 # number of agents
    num_episode = 1    # Number of total episode in a single simulation for taking ensemble average
    time_steps =30
    threshold = 1/2
    Z_func = "A" # the next strategy if number of A-playing neighbors equlas B-playing ones
    always_equilibrates = 1
    global new_result
    result = pd.DataFrame({'reached_desired?': [],'Equilibrated?': [],'types': [],'initial strategy set': [],'final strategy set': [],'equilibration time': []})
    # algorithm does not work on(pop=9): (loops :))
    desired_eq = ['A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'A'] #desired equilibrium strategy of the agents
    type_list = ['+', '+', '-', '-', '-', '+', '-', '-', '-'] #coordinator or anticoordinator listed identified by + and - signs
    initial_strategy  =['B', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A'] #initial strategy of the agents
    #it works on: (original example ;))
#     desired_eq = ['A', 'B', 'A', 'A', 'B', 'A', 'A', 'A']  # desired equilibrium strategy of the agents
#     type_list = ['+', '-', '-', '+', '-', '-', '+', '+']  # coordinator or anticoordinator listed identified by + and - signs
#     initial_strategy = ['B', 'A', 'B', 'A', 'A', 'B', 'A','B']  # initial strategy of the agents# selection = [i for i in range(population)]

    # selection = [i for i in range(population)]
    activated_list = zeros(population)
    simulation = Simulation(population, Z_func, initial_strategy, type_list,activated_list, desired_eq)
    new_result,equilibrated,eq_time,reached_desired = simulation.one_episode(time_steps, threshold)
#     result = pd.DataFrame.from_records
    result = result.append(new_result)
    result.to_csv(f"newfile.csv")


if __name__ == '__main__':
    main()


c1: [] c2: [1, 6] c3: [0, 4, 7]
agents activated so far: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
in timestep 0 selected: agent 1
selected agent : 1
c1: [] c2: [2, 6] c3: [0, 4, 7]
agents activated so far: [0. 1. 0. 0. 0. 0. 0. 0. 0.]
in timestep 1 selected: agent 2
selected agent : 2
c1: [] c2: [3, 6] c3: [0, 1, 4, 7]
agents activated so far: [0. 1. 1. 0. 0. 0. 0. 0. 0.]
in timestep 2 selected: agent 3
selected agent : 3
c1: [] c2: [6] c3: [0, 1, 7]
agents activated so far: [0. 1. 1. 1. 0. 0. 0. 0. 0.]
in timestep 3 selected: agent 6
selected agent : 6
c1: [] c2: [] c3: [0, 1]
agents activated so far: [0. 1. 1. 1. 0. 0. 1. 0. 0.]
in timestep 4 selected: agent 0
selected agent : 0
c1: [] c2: [8] c3: [1]
agents activated so far: [1. 1. 1. 1. 0. 0. 1. 0. 0.]
in timestep 5 selected: agent 8
selected agent : 8
c1: [] c2: [0] c3: [1]
agents activated so far: [1. 1. 1. 1. 0. 0. 1. 0. 1.]
in timestep 6 selected: agent 0
selected agent : 0
c1: [8] c2: [] c3: [1]
agents activated so far: [2. 1. 1. 1. 0. 0.

/tmp/ipykernel_4017/3260953646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_result)
